# Convert COAWST model data from NetCDF to Zarr

In [1]:
import xarray as xr
import glob

from dask.distributed import Client
from dask_jobqueue import SLURMCluster

In [3]:
cluster = SLURMCluster(processes=1, cores=30, memory='6GB', interface='ipogif0',
                       project='woodshole', walltime='01:00:00')

In [4]:
client = Client(cluster)
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -A woodshole
#SBATCH -n 1
#SBATCH --cpus-per-task=30
#SBATCH --mem=6G
#SBATCH -t 02:00:00

/home/rsignell/miniconda3/envs/pangeo/bin/python -m distributed.cli.dask_worker tcp://10.128.0.39:39379 --nthreads 30 --memory-limit 6.00GB --name name --nanny --death-timeout 60 --interface ipogif0



In [5]:
cluster.scale(10)

In [6]:
cluster

In [7]:
flist = glob.glob('/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_*.nc')

In [8]:
flist.sort()

In [9]:
flist

['/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_00001.nc',
 '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_00002.nc',
 '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_00003.nc',
 '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_00004.nc',
 '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_00005.nc',
 '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_00006.nc',
 '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_00007.nc',
 '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_

In [10]:
zarr_ds = '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/zarr_001b'

In [11]:
for file in flist:
    print('starting '+file.split('/')[-1])
    ds = xr.open_dataset(file, chunks={'ocean_time':1})
    if '00001' in file:
        ds.to_zarr(zarr_ds, consolidated=True, mode='w')
    else:
        ds.to_zarr(zarr_ds, consolidated=True, append_dim='ocean_time')
    print('finished '+file.split('/')[-1])

starting ocean_deb_his_run07A_zo015_00001.nc
finished ocean_deb_his_run07A_zo015_00001.nc
starting ocean_deb_his_run07A_zo015_00002.nc
finished ocean_deb_his_run07A_zo015_00002.nc
starting ocean_deb_his_run07A_zo015_00003.nc
finished ocean_deb_his_run07A_zo015_00003.nc
starting ocean_deb_his_run07A_zo015_00004.nc
finished ocean_deb_his_run07A_zo015_00004.nc
starting ocean_deb_his_run07A_zo015_00005.nc
finished ocean_deb_his_run07A_zo015_00005.nc
starting ocean_deb_his_run07A_zo015_00006.nc
finished ocean_deb_his_run07A_zo015_00006.nc
starting ocean_deb_his_run07A_zo015_00007.nc
finished ocean_deb_his_run07A_zo015_00007.nc
starting ocean_deb_his_run07A_zo015_00008.nc
finished ocean_deb_his_run07A_zo015_00008.nc
starting ocean_deb_his_run07A_zo015_00009.nc
finished ocean_deb_his_run07A_zo015_00009.nc
starting ocean_deb_his_run07A_zo015_00010.nc
finished ocean_deb_his_run07A_zo015_00010.nc
starting ocean_deb_his_run07A_zo015_00011.nc
finished ocean_deb_his_run07A_zo015_00011.nc
starting o

In [12]:
xr.open_zarr(zarr_ds, consolidated=True)

<xarray.Dataset>
Dimensions:          (boundary: 4, eta_psi: 1369, eta_rho: 1370, eta_u: 1370, eta_v: 1369, ocean_time: 8755, s_rho: 16, s_w: 17, tracer: 2, xi_psi: 183, xi_rho: 184, xi_u: 183, xi_v: 184)
Coordinates:
    lat_psi          (eta_psi, xi_psi) float64 dask.array<chunksize=(1369, 183), meta=np.ndarray>
    lat_rho          (eta_rho, xi_rho) float64 dask.array<chunksize=(1370, 184), meta=np.ndarray>
    lat_u            (eta_u, xi_u) float64 dask.array<chunksize=(1370, 183), meta=np.ndarray>
    lat_v            (eta_v, xi_v) float64 dask.array<chunksize=(1369, 184), meta=np.ndarray>
    lon_psi          (eta_psi, xi_psi) float64 dask.array<chunksize=(1369, 183), meta=np.ndarray>
    lon_rho          (eta_rho, xi_rho) float64 dask.array<chunksize=(1370, 184), meta=np.ndarray>
    lon_u            (eta_u, xi_u) float64 dask.array<chunksize=(1370, 183), meta=np.ndarray>
    lon_v            (eta_v, xi_v) float64 dask.array<chunksize=(1369, 184), meta=np.ndarray>
  * ocean_time       (ocean_time) datetime64[ns] 2017-01-01 ... 2017-12-31T18...
  * s_rho            (s_rho) float64 -0.9688 -0.9062 ... -0.09375 -0.03125
  * s_w              (s_w) float64 -1.0 -0.9375 -0.875 ... -0.125 -0.0625 0.0
Dimensions without coordinates: boundary, eta_psi, eta_rho, eta_u, eta_v, tracer, xi_psi, xi_rho, xi_u, xi_v
Data variables:
    AKv              (ocean_time, s_w, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 17, 1370, 184), meta=np.ndarray>
    Akk_bak          float64 ...
    Akp_bak          float64 ...
    Akt_bak          (tracer) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    Akv_bak          float64 ...
    Charnok_alpha    float64 ...
    CrgBan_cw        float64 ...
    Cs_r             (s_rho) float64 dask.array<chunksize=(16,), meta=np.ndarray>
    Cs_w             (s_w) float64 dask.array<chunksize=(17,), meta=np.ndarray>
    FSobc_in         (boundary) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    FSobc_out        (boundary) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    Falpha           float64 ...
    Fbeta            float64 ...
    Fgamma           float64 ...
    Lm2CLM           int32 ...
    Lm3CLM           int32 ...
    LnudgeM2CLM      int32 ...
    LnudgeM3CLM      int32 ...
    LnudgeTCLM       (tracer) int32 dask.array<chunksize=(2,), meta=np.ndarray>
    LsshCLM          int32 ...
    LtracerCLM       (tracer) int32 dask.array<chunksize=(2,), meta=np.ndarray>
    LtracerSponge    (tracer) int32 dask.array<chunksize=(2,), meta=np.ndarray>
    LtracerSrc       (tracer) int32 dask.array<chunksize=(2,), meta=np.ndarray>
    LuvSponge        int32 ...
    LuvSrc           int32 ...
    LwSrc            int32 ...
    M2nudg           float64 ...
    M2obc_in         (boundary) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    M2obc_out        (boundary) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    M3nudg           float64 ...
    M3obc_in         (boundary) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    M3obc_out        (boundary) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    R0               float64 ...
    Scoef            float64 ...
    Tcline           float64 ...
    Tcoef            float64 ...
    Tnudg            (tracer) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    Tnudg_SSS        float64 ...
    Tobc_in          (boundary, tracer) float64 dask.array<chunksize=(4, 2), meta=np.ndarray>
    Tobc_out         (boundary, tracer) float64 dask.array<chunksize=(4, 2), meta=np.ndarray>
    Vstretching      int32 ...
    Vtransform       int32 ...
    Znudg            float64 ...
    Zob              float64 ...
    Zos              float64 ...
    Zos_hsig_alpha   float64 ...
    angle            (eta_rho, xi_rho) float64 dask.array<chunksize=(1370, 184), meta=np.ndarray>
    bustr            (ocean_time, eta_u, xi_u) float32 dask.array<chunksize=(1, 1370, 183), meta=np.ndarray>
    bvstr            (ocean_time, eta_v, xi_v) floa

In [ ]:
#cluster.close()